In [1]:
from sklearn.datasets import load_digits
digits_features, digits_labels = load_digits(return_X_y=True)
print("Features shape: {}\n".format(digits_features.shape))
print("Labels shape: {}\n".format(digits_labels.shape))

Features shape: (1797, 64)

Labels shape: (1797,)



In [2]:
import numpy as np
mean_vals = np.mean(digits_features, axis=0)
std_val = np.std(digits_features)

digits_features_std = (digits_features - mean_vals)/std_val

In [3]:
import tensorflow as tf
import tensorflow.keras as keras

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    digits_features_std, 
    digits_labels, 
    test_size=0.2, 
    random_state=1
)
print("Training features shape: {}\n".format(X_train.shape))
print("Training labels shape: {}\n".format(y_train.shape))
print("Test features shape: {}\n".format(X_test.shape))
print("Test labels shape: {}\n".format(y_test.shape))

Training features shape: (1437, 64)

Training labels shape: (1437,)

Test features shape: (360, 64)

Test labels shape: (360,)



In [5]:
y_train_one_hot = keras.utils.to_categorical(y_train) 
y_test_one_hot = keras.utils.to_categorical(y_test) 
print("One-hot encoded labels shape: {} - {}\n".format(y_train_one_hot.shape, y_test_one_hot.shape))



One-hot encoded labels shape: (1437, 10) - (360, 10)



In [8]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
random_seed = 1

g = tf.Graph()
with g.as_default():
    tf.set_random_seed(random_seed)
    tf_x = tf.placeholder(dtype=tf.float32, shape=(None, X_train.shape[1]), 
                          name='tf_x')
    tf_y = tf.placeholder(dtype = tf.int32, shape=None, name='tf_y')
    y_one_hot = tf.one_hot(indices= tf_y, depth = 10)
    
    h1 = tf.layers.dense(inputs=tf_x, units=50, activation=tf.tanh, name='layer1')
    h2 = tf.layers.dense(inputs=h1, units=50, activation=tf.tanh, name='layer2')
    logits = tf.layers.dense(inputs=h2, units=10, activation=None, name='layer3')
    predictions = {
        'classes' : tf.argmax(logits, axis=1, name='predicted_classes'),
        'probabilities' : tf.nn.softmax(logits, name='softmax_tensor')
    }
    
    cost = tf.losses.softmax_cross_entropy(
        onehot_labels=y_one_hot, logits=logits)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(loss=cost)
    init_op = tf.global_variables_initializer()
    

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [9]:
def create_batch_generator(X, y, batch_size=128, shuffle=False):
    X_copy = np.array(X)
    y_copy = np.array(y)

    for i in range(0, X.shape[0], batch_size):
        yield (X_copy[i:i+batch_size, :], y_copy[i:i+batch_size])

In [10]:
## create a session to launch the graph
sess = tf.Session(graph=g)
sess.run(init_op)
for epoch in range(50):
    training_costs = []
    batch_generator = create_batch_generator(
        X_train, 
        y_train,
        batch_size=64)
    for batch_X, batch_y in batch_generator:
        feed = {tf_x:batch_X, tf_y:batch_y}
        _, batch_cost = sess.run([train_op, cost], feed_dict=feed)
        training_costs.append(batch_cost)
        print(' -- Epoch %2d Avg. Training Loss: %.4f' % (
            epoch+1, 
            np.mean(training_costs))
        )

 -- Epoch  1 Avg. Training Loss: 2.3173
 -- Epoch  1 Avg. Training Loss: 2.3772
 -- Epoch  1 Avg. Training Loss: 2.3713
 -- Epoch  1 Avg. Training Loss: 2.3907
 -- Epoch  1 Avg. Training Loss: 2.4170
 -- Epoch  1 Avg. Training Loss: 2.4020
 -- Epoch  1 Avg. Training Loss: 2.3951
 -- Epoch  1 Avg. Training Loss: 2.4114
 -- Epoch  1 Avg. Training Loss: 2.4245
 -- Epoch  1 Avg. Training Loss: 2.4325
 -- Epoch  1 Avg. Training Loss: 2.4424
 -- Epoch  1 Avg. Training Loss: 2.4453
 -- Epoch  1 Avg. Training Loss: 2.4473
 -- Epoch  1 Avg. Training Loss: 2.4545
 -- Epoch  1 Avg. Training Loss: 2.4619
 -- Epoch  1 Avg. Training Loss: 2.4664
 -- Epoch  1 Avg. Training Loss: 2.4704
 -- Epoch  1 Avg. Training Loss: 2.4662
 -- Epoch  1 Avg. Training Loss: 2.4628
 -- Epoch  1 Avg. Training Loss: 2.4643
 -- Epoch  1 Avg. Training Loss: 2.4670
 -- Epoch  1 Avg. Training Loss: 2.4688
 -- Epoch  1 Avg. Training Loss: 2.4738
 -- Epoch  2 Avg. Training Loss: 2.2744
 -- Epoch  2 Avg. Training Loss: 2.3350


In [11]:
feed = {tf_x : X_test}
y_pred = sess.run(predictions['classes'],feed_dict=feed)
print('Test Accuracy: %.2f%%' % (
        100*np.sum(y_pred == y_test)/y_test.shape[0]))


Test Accuracy: 75.00%
